## Day 5 - Evaluation

In [3]:
import io
import zipfile
import requests
import frontmatter

doc_extensions = {'md', 'mdx'}
code_extensions = {'py', 'sql', 'java', 'ipynb'}
extensions = doc_extensions | code_extensions

def read_repo_data(repo_owner, repo_name):
    """
    Download and parse all markdown and code files from a GitHub repository.
    
    Args:
        repo_owner: GitHub username or organization
        repo_name: Repository name
    
    Returns:
        List of dictionaries containing file content and metadata
    """ 
    url = f'https://github.com/{repo_owner}/{repo_name}/archive/refs/heads/main.zip'
    resp = requests.get(url)
    
    if resp.status_code != 200:
        raise Exception(f"Failed to download repository: {resp.status_code}")

    repository_data = []
    zf = zipfile.ZipFile(io.BytesIO(resp.content))
    
    for file_info in zf.infolist():
        filepath = file_info.filename
        filepath_lower = filepath.lower()

        if filepath_lower.endswith('/'):
            continue

        filename = filepath_lower.split('/')[-1]

        if filename.startswith('.'):
            continue

        ext = filename.split('.')[-1]

        if ext not in extensions:
            continue

        filepath_edited = filepath.split('/', maxsplit=1)[1]

        try:
            with zf.open(file_info) as f_in:
                content = f_in.read().decode('utf-8', errors='ignore')
                if ext in doc_extensions:
                    post = frontmatter.loads(content)
                    data = post.to_dict()
                    data['filename'] = filepath_edited
                elif ext in code_extensions:
                    data = {
                        'code': True,
                        'content': content,
                        'filename': filepath_edited
                    }

                repository_data.append(data)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

    zf.close()
    return repository_data

In [5]:
import json
from minsearch import Index, VectorSearch
from sentence_transformers import SentenceTransformer
from typing import List, Any
import numpy as np
from tqdm.auto import tqdm # Import tqdm for progress bar

from dotenv import load_dotenv
from openai import OpenAI

# This line loads the variables from your .env file
load_dotenv()
openai_client = OpenAI()

# Assuming read_repo_data is defined elsewhere
# (Note: This is the data loading step)
autogen_data = read_repo_data('microsoft', 'autogen')

def sliding_window(seq, size, step):
    """Chunks text into overlapping sections."""
    if size <= 0 or step <= 0:
        raise ValueError("size and step must be positive")
    n = len(seq)
    result = []
    for i in range(0, n, step):
        chunk_content = seq[i:i+size]
        # Stores the chunked text under the 'content' key
        result.append({'start': i, 'content': chunk_content}) 
        if i + size >= n:
            break
    return result

# --- Data Preparation and Chunking ---
autogen_data_chunks = []

for doc in autogen_data:
    doc_copy = doc.copy()
    doc_content = doc_copy.pop('content')
    chunks = sliding_window(doc_content, 2000, 1000)
    for chunk in chunks:
        # Merge chunk content with document metadata (filename, etc.)
        chunk.update(doc_copy) 
    autogen_data_chunks.extend(chunks)

# --- 1. Text Search Index (minsearch Index) ---
# FIX: Define and fit the text index
aut_index = Index(
    text_fields=["filename", "content"],
    keyword_fields=[]
)
aut_index.fit(autogen_data_chunks)

# --- 2. Vector Search Index (minsearch VectorSearch) ---
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')
autogen_embeddings = []

# FIX: Use the correct list name: autogen_data_chunks
for d in tqdm(autogen_data_chunks):
    # d['content'] is the correct key holding the chunked text
    v = embedding_model.encode(d['content'])
    autogen_embeddings.append(v)

autogen_embeddings = np.array(autogen_embeddings)

autogen_vindex = VectorSearch()
# FIX: Use the correct list name: autogen_data_chunks
autogen_vindex.fit(autogen_embeddings, autogen_data_chunks)


# --- HYBRID SEARCH FUNCTION (The Agent's Tool) ---
def hybrid_search(query: str) -> List[Any]:
    """
    Performs a Hybrid Search combining Text (Keyword) and Vector (Semantic) search.
    This is the function the agent will call.
    """
    # 1. Text Search (Keyword matching)
    text_results = aut_index.search(query, num_results=5)
    
    # 2. Vector Search (Semantic matching)
    q_vector = embedding_model.encode(query)
    vector_results = autogen_vindex.search(q_vector, num_results=5)
    
    # 3. Combine and Deduplicate Results
    seen_filenames = set()
    combined_results = []

    for result in text_results + vector_results:
        # Deduplicate based on 'filename'
        if result['filename'] not in seen_filenames:
            seen_filenames.add(result['filename'])
            # Return only the essential information for the LLM
            combined_results.append({
                "filename": result['filename'],
                "content": result['content']
            })

    return combined_results


# --- AGENT SETUP AND EXECUTION ---

system_prompt = """
You are a helpful assistant for the AutoGen course.
Your sole purpose is to answer questions using the 'hybrid_search' tool, which queries the official AutoGen documentation.
Always use the tool before answering.
"""

# Assuming pydantic_ai and Agent are defined/imported correctly
from pydantic_ai import Agent

agent = Agent(
    name="autogen_agent",
    instructions=system_prompt,
    # FIX: Pass the HYBRID search function as the tool
    tools=[hybrid_search],
    model='gpt-4o-mini'
)

# Use the specific, enforced question to trigger the tool call
question = 'what checks must be met to run a PR in the **AutoGen** repository?'

result = await agent.run(user_prompt=question)
print(result)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 54199528-99cf-43df-8073-4f983942b952)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/multi-qa-distilbert-cos-v1/resolve/main/1_Pooling/config.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 099f5682-af53-45e1-bb50-f99309b0efd7)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/multi-qa-distilbert-cos-v1/resolve/main/1_Pooling/config.json
Retrying in 2s [Retry 2/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/multi-qa-distilbert-cos-v1/resolve/main/1_Pooling/config.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x78778819b8b0>: Failed to establish a new connection: [Errno 101] Network i

  0%|          | 0/6951 [00:00<?, ?it/s]

AgentRunResult(output='To run a pull request (PR) in the AutoGen repository, the following checks must be met:\n\n1. **Documentation Changes**: Ensure that any necessary documentation changes have been included, as outlined in the documentation link [here](https://microsoft.github.io/autogen/). You should check the contributing guidelines to build and test documentation locally as specified.\n\n2. **Test Coverage**: If relevant, tests should be added that correspond to the changes introduced in the PR.\n\n3. **Auto Checks**: Make sure that all the automatic checks have passed. This typically includes validations for code style, tests, and other quality checks established by the repository.\n\n4. **Add a Reviewer**: When you create a PR, add a reviewer to the assignee section; if you do not have the access to do so, a reviewer will be assigned shortly.\n\nIt is valuable to note that you should ensure that all projects compile and run tests successfully as part of the validations before 

#### LOGGING

what we want to record:
* The system prompt that we used
* The model
* The user query
* The tools we use
* The responses and the back-and-forth interactions between the LLM and our tools
* The final response


In [6]:
# create log

from pydantic_ai.messages import ModelMessagesTypeAdapter


def log_entry(agent, messages, source="user"):
    tools = []

    for ts in agent.toolsets:
        tools.extend(ts.tools.keys())

    dict_messages = ModelMessagesTypeAdapter.dump_python(messages)

    return {
        "agent_name": agent.name,
        "system_prompt": agent._instructions,
        "provider": agent.model.system,
        "model": agent.model.model_name,
        "tools": tools,
        "messages": dict_messages,
        "source": source
    }

In [7]:
# write log to a folder
import secrets
from pathlib import Path
from datetime import datetime


LOG_DIR = Path('logs')
LOG_DIR.mkdir(exist_ok=True)


def serializer(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError(f"Type {type(obj)} not serializable")


def log_interaction_to_file(agent, messages, source='user'):
    entry = log_entry(agent, messages, source)

    ts = entry['messages'][-1]['timestamp']
    ts_str = ts.strftime("%Y%m%d_%H%M%S")
    rand_hex = secrets.token_hex(3)

    filename = f"{agent.name}_{ts_str}_{rand_hex}.json"
    filepath = LOG_DIR / filename

    with filepath.open("w", encoding="utf-8") as f_out:
        json.dump(entry, f_out, indent=2, default=serializer)

    return filepath

In [8]:
question = input()
result = await agent.run(user_prompt=question)
print(result.output)
log_interaction_to_file(agent, result.new_messages())

 must i update the dependencies whenever i pull new changes?


Yes, it is generally recommended to update your dependencies whenever you pull new changes, especially if those changes involve updates to the packages or libraries that your project depends on. This ensures that you are working with the latest features, improvements, and security patches.

In the context of AutoGen, you might want to specify the version of AutoGen in your dependency management files (like `pyproject.toml` for Python) to ensure compatibility. For example:

```toml
[project]
# ...
dependencies = [
    "autogen-core>=0.4,<0.5"
]
```

This kind of version specification helps ensure that your extensions and applications integrate properly with the version of AutoGen you are using. Regularly updating your dependencies alongside pulling the latest changes can prevent compatibility issues and allow you to leverage new features and fixes.


PosixPath('logs/autogen_agent_20250928_171138_46accd.json')

In [9]:
question = input()
result = await agent.run(user_prompt=question)
print(result.output)
log_interaction_to_file(agent, result.new_messages())

 okay, how can i update the dependencies using uv


To update the dependencies using the `uv` tool, you can follow these steps:

1. **Activate your Virtual Environment**: Make sure you are in the virtual environment where your project dependencies are managed.
2. **Run the Update Command**: In the directory of your Python project, execute the following command:

   ```sh
   uv sync --all-extras
   ```

This command will synchronize and update all the dependencies defined in your project.


PosixPath('logs/autogen_agent_20250928_171321_e3a625.json')

In [10]:
question = input()
result = await agent.run(user_prompt=question)
print(result.output)
log_interaction_to_file(agent, result.new_messages())

 okay. where is the documentation source directory?


The documentation source directory for the AutoGen project is located at `docs/src/`. You can build the documentation by running the command `poe docs-build` from the root of the Python directory.


PosixPath('logs/autogen_agent_20250928_171450_0ddd80.json')

write these logs to a folder:

### Adding References
When interacting with the agent, I noticed one thing: it doesn't include the reference to the original documents.
Let's fix it by adjusting the prompt:


In [13]:
system_prompt = """
You are a helpful assistant for a course.  

Use the search tool to find relevant information from the course materials before answering questions.  

If you can find specific information through search, use it to provide accurate answers.

Always include references by citing the filename of the source material you used.  
When citing the reference, replace "faq-main" by the full path to the GitHub repository: "https://github.com/DataTalksClub/faq/blob/main/"
Format: [LINK TITLE](FULL_GITHUB_LINK)

If the search doesn't return relevant results, let the user know and provide general guidance.  
""".strip()

# Create another version of agent, let's call it faq_agent_v2
agent = Agent(
    name="aut_agent_v2",
    instructions=system_prompt,
    tools=[hybrid_search],
    model='gpt-4o-mini'
)


In [14]:
question = input()
result = await agent.run(user_prompt=question)
print(result.output)
log_interaction_to_file(agent, result.new_messages())

 where is the documentation source directory?


The documentation source directory is located at `docs/src/`. This directory contains the source files for the documentation which is built using the Sphinx documentation system.

To build the documentation, you can run `poe docs-build` from the root of the Python directory. If you need to serve the documentation locally, use the command `poe docs-serve`.

For further details on building and maintaining the documentation, refer to [this README](https://github.com/DataTalksClub/faq/blob/main/python/README.md).


PosixPath('logs/aut_agent_v2_20250928_172003_561fe7.json')

### LLM as a Judge

In [16]:
evaluation_prompt = """
Use this checklist to evaluate the quality of an AI agent's answer (<ANSWER>) to a user question (<QUESTION>).
We also include the entire log (<LOG>) for analysis.

For each item, check if the condition is met. 

Checklist:

- instructions_follow: The agent followed the user's instructions (in <INSTRUCTIONS>)
- instructions_avoid: The agent avoided doing things it was told not to do  
- answer_relevant: The response directly addresses the user's question  
- answer_clear: The answer is clear and correct  
- answer_citations: The response includes proper citations or sources when required  
- completeness: The response is complete and covers all key aspects of the request
- tool_call_search: Is the search tool invoked? 

Output true/false for each check and provide a short explanation for your judgment.
""".strip()

Since we expect a very well defined structure of the response, we can use structured output.
We can define a Pydantic class with the expected response structure, and the LLM will produce output that matches this schema exactly.
This is how we do it:


In [18]:
from pydantic import BaseModel

class EvaluationCheck(BaseModel):
    check_name: str
    justification: str
    check_pass: bool

class EvaluationChecklist(BaseModel):
    checklist: list[EvaluationCheck]
    summary: str

In [19]:
eval_agent = Agent(
    name='eval_agent',
    model='gpt-5-nano', # different gpt for the evaluation
    instructions=evaluation_prompt,
    output_type=EvaluationChecklist
)


In [21]:
# input template
user_prompt_format = """
<INSTRUCTIONS>{instructions}</INSTRUCTIONS>
<QUESTION>{question}</QUESTION>
<ANSWER>{answer}</ANSWER>
<LOG>{log}</LOG>
""".strip()

In [22]:
#define a helper function for loading the JSON log files

def load_log_file(log_file):
    with open(log_file, 'r') as f_in:
        log_data = json.load(f_in)
        log_data['log_file'] = log_file
        return log_data

In [23]:
# We also add the filename in the result - it'll help us with tracking later.

log_record = load_log_file('./logs/aut_agent_v2_20250928_172003_561fe7.json')

instructions = log_record['system_prompt']
question = log_record['messages'][0]['parts'][0]['content']
answer = log_record['messages'][-1]['parts'][0]['content']
log = json.dumps(log_record['messages'])

user_prompt = user_prompt_format.format(
    instructions=instructions,
    question=question,
    answer=answer,
    log=log
)


In [24]:
result = await eval_agent.run(user_prompt, output_type=EvaluationChecklist)

checklist = result.output
print(checklist.summary)

for check in checklist.checklist:
    print(check)

Answer provided with the documentation directory path and a source citation. A dedicated search tool was not invoked in this interaction, but the answer aligns with course materials.
check_name='instructions_follow' justification='The assistant provided a direct answer based on course materials and cited a source as requested.' check_pass=True
check_name='instructions_avoid' justification="No disallowed actions were performed; the answer is focused on the user's question." check_pass=True
check_name='answer_relevant' justification='The answer directly states the location of the documentation source directory.' check_pass=True
check_name='answer_clear' justification='The response is concise and clear.' check_pass=True
check_name='answer_citations' justification='Citations are provided with the filename and full GitHub link as instructed.' check_pass=True
check_name='completeness' justification="Addresses the user's query with the exact directory path and a source reference." check_pass=

Note that we're putting the entire conversation log into the prompt, which is not really necessary. We can reduce it to make it less verbose.
For example, like that:


In [25]:
def simplify_log_messages(messages):
    log_simplified = []

    for m in messages:
        parts = []
    
        for original_part in m['parts']:
            part = original_part.copy()
            kind = part['part_kind']
    
            if kind == 'user-prompt':
                del part['timestamp']
            if kind == 'tool-call':
                del part['tool_call_id']
            if kind == 'tool-return':
                del part['tool_call_id']
                del part['metadata']
                del part['timestamp']
                # Replace actual search results with placeholder to save tokens
                part['content'] = 'RETURN_RESULTS_REDACTED'
            if kind == 'text':
                del part['id']
    
            parts.append(part)
    
        message = {
            'kind': m['kind'],
            'parts': parts
        }
    
        log_simplified.append(message)
    return log_simplified

#### PUT EVERYTHING TOGETHER

In [26]:
async def evaluate_log_record(eval_agent, log_record):
    messages = log_record['messages']

    instructions = log_record['system_prompt']
    question = messages[0]['parts'][0]['content']
    answer = messages[-1]['parts'][0]['content']

    log_simplified = simplify_log_messages(messages)
    log = json.dumps(log_simplified)

    user_prompt = user_prompt_format.format(
        instructions=instructions,
        question=question,
        answer=answer,
        log=log
    )

    result = await eval_agent.run(user_prompt, output_type=EvaluationChecklist)
    return result.output 


log_record = load_log_file('./logs/aut_agent_v2_20250928_172003_561fe7.json')
eval1 = await evaluate_log_record(eval_agent, log_record)

In [27]:
# Print the summary property
print(eval1.summary) 

Answer will present the docs/src directory location and provide a citable reference to python/README.md on GitHub.


In [28]:
for check in eval1.checklist:
    print(check)

check_name='instructions_follow' justification="The answer will include a citation and directly address the location of the documentation source directory as requested, following the user's instruction to cite source material." check_pass=True
check_name='instructions_avoid' justification='No prohibited content; concise guidance on location and simple commands is provided.' check_pass=True
check_name='answer_relevant' justification='Directly answers where the documentation source directory is located.' check_pass=True
check_name='answer_clear' justification='Answer is concise and clear about the path and actions to build/serve.' check_pass=True
check_name='answer_citations' justification='Includes a citation to the source material using the requested GitHub link format.' check_pass=True
check_name='completeness' justification='Provides location, basic build/serve commands, and a reference for more details.' check_pass=True
check_name='tool_call_search' justification='A tool call was is

### Data Generation

In [42]:
question_generation_prompt = """
You are helping to create test questions for an AI agent that answers questions about the AutoGen documentation.

Based on the provided content (which is a list of documentation chunks), generate realistic questions that students might ask.

The questions should:

- Be natural and varied in style
- Range from simple to complex
- Include both specific technical questions and general usage questions
- **Generate exactly 10 questions in total, ensuring each question is grounded in the provided content.**
""".strip()

class QuestionsList(BaseModel):
    questions: list[str]

question_generator = Agent(
    name="question_generator",
    instructions=question_generation_prompt,
    model='gpt-4o-mini',
    output_type=QuestionsList
)


let's sample 10 records from our dataset using Python's built-in random.sample function:


In [43]:
import random

sample = random.sample(autogen_data, 10)
prompt_docs = [d['content'] for d in sample]
prompt = json.dumps(prompt_docs)

result = await question_generator.run(prompt)
questions = result.output.questions

In [44]:
print(questions)

['What environment variables need to be set for the `google_search` function to work?', 'Can you explain how the `fetch_page_content` helper function works within the `google_search` function?', 'What are the default parameters for the `google_search` function, and what do they represent?', 'How do you create an `AssistantAgent` using an OpenAI model?', 'What happens if the API key or CSE ID is missing when attempting to make a Google search?', 'How does the `LangChainToolAdapter` class facilitate the use of LangChain tools in AutoGen?', 'What is the purpose of the `TokenLimitedChatCompletionContext` component in AutoGen?', 'Can you describe how audio extraction and transcription are handled in the provided video processing code?', 'What changes should I make in the code if I want to enable safe search when using the `google_search` function?', 'How do you handle exceptions when making requests with the `httpx` client in the `google_search` function?']


In [45]:
from tqdm.auto import tqdm

for q in tqdm(questions):
    print(q)

    result = await agent.run(user_prompt=q)
    print(result.output)

    log_interaction_to_file(
        agent,
        result.new_messages(),
        source='ai-generated'
    )

    print()

  0%|          | 0/10 [00:00<?, ?it/s]

What environment variables need to be set for the `google_search` function to work?
To use the `google_search` function, you need to set the following environment variables:

1. `GOOGLE_API_KEY`: This is your Google API key that grants access to the Google Custom Search API.
2. `GOOGLE_SEARCH_ENGINE_ID`: This is the ID of your Google Custom Search Engine (CSE) that you want to use for the searches.

If either of these variables is not set, an error will be raised indicating that the API key or Search Engine ID is not found in the environment variables.

For further details, you can refer to the source material: [google_search.py](https://github.com/DataTalksClub/faq/blob/main/python/packages/autogen-studio/autogenstudio/gallery/tools/google_search.py).

Can you explain how the `fetch_page_content` helper function works within the `google_search` function?
The `fetch_page_content` helper function is utilized within the `google_search` function to retrieve the content of a web page given

In [46]:
#First, collect all the AI-generated logs for the v2 agent:

eval_set = []

for log_file in LOG_DIR.glob('*.json'):
    if 'aut_agent_v2' not in log_file.name:
        continue

    log_record = load_log_file(log_file)
    if log_record['source'] != 'ai-generated':
        continue

    eval_set.append(log_record)

In [47]:
# evaluate them:

eval_results = []

for log_record in tqdm(eval_set):
    eval_result = await evaluate_log_record(eval_agent, log_record)
    eval_results.append((log_record, eval_result))

  0%|          | 0/11 [00:00<?, ?it/s]

In [48]:
# transform the rsults to a form so that pandas dataframe can handle it
rows = []

for log_record, eval_result in eval_results:
    messages = log_record['messages']

    row = {
        'file': log_record['log_file'].name,
        'question': messages[0]['parts'][0]['content'],
        'answer': messages[-1]['parts'][0]['content'],
    }

    checks = {c.check_name: c.check_pass for c in eval_result.checklist}
    row.update(checks)

    rows.append(row)

In [49]:
import pandas as pd

df_evals = pd.DataFrame(rows)

In [50]:
df_evals

,file,question,answer,instructions_follow,instructions_avoid,answer_relevant,answer_clear,answer_citations,completeness,tool_call_search
0,aut_agent_v2_20250928_175535_afbcba.json,What is the role of the WorkerAgent class in h...,The `WorkerAgent` class functions as an essent...,True,True,True,True,True,True,False
1,aut_agent_v2_20250928_181142_ba5108.json,What environment variables need to be set for ...,"To use the `google_search` function, you need ...",True,True,True,True,True,True,True
2,aut_agent_v2_20250928_181145_9f3d2b.json,Can you explain how the `fetch_page_content` h...,The `fetch_page_content` helper function is ut...,True,True,True,True,True,True,False
3,aut_agent_v2_20250928_181153_2b8619.json,What are the default parameters for the `googl...,The `google_search` function has the following...,True,True,True,True,True,True,True
4,aut_agent_v2_20250928_181206_5d5492.json,How do you create an `AssistantAgent` using an...,To create an `AssistantAgent` using an OpenAI ...,True,True,True,True,True,True,True
5,aut_agent_v2_20250928_181213_79be66.json,What happens if the API key or CSE ID is missi...,If the API key or the Custom Search Engine (CS...,True,True,True,True,True,True,True
6,aut_agent_v2_20250928_181219_23bac2.json,How does the `LangChainToolAdapter` class faci...,The `LangChainToolAdapter` class facilitates t...,True,True,True,True,True,True,False
7,aut_agent_v2_20250928_181228_4dfc20.json,What is the purpose of the `TokenLimitedChatCo...,The `TokenLimitedChatCompletionContext` compon...,False,True,True,True,False,True,False
8,aut_agent_v2_20250928_181236_65c9a6.json,Can you describe how audio extraction and tran...,"In the provided video processing code, audio e...",True,True,True,True,False,True,True
9,aut_agent_v2_20250928_181244_acc083.json,What changes should I make in the code if I wa...,To enable safe search when using the `google_s...,False,True,True,True,True,True,False


In [51]:
df_evals.mean(numeric_only=True)

instructions_follow    0.818182
instructions_avoid     1.000000
answer_relevant        1.000000
answer_clear           1.000000
answer_citations       0.818182
completeness           1.000000
tool_call_search       0.545455
dtype: float64

### Evaluating functions and tools

In [41]:
def evaluate_search_quality(search_function, test_queries):
    results = []
    
    for query, expected_docs in test_queries:
        search_results = search_function(query, num_results=5)
        
        # Calculate hit rate
        relevant_found = any(doc['filename'] in expected_docs for doc in search_results)
        
        # Calculate MRR
        for i, doc in enumerate(search_results):
            if doc['filename'] in expected_docs:
                mrr = 1 / (i + 1)
                break
        else:
            mrr = 0
            
        results.append({
            'query': query,
            'hit': relevant_found,
            'mrr': mrr
        })
    return results